Part 3 of the LLM per game TLDR generation project

Given the processed game reviews, we create different tables like in GameAnalysis API

In [30]:
from datetime import datetime
import pytz
import os
import time
import requests
import json
import sys

import pickle
from pathlib import Path
import traceback

import pandas as pd
import numpy as np

In [31]:
# game_steamid = 1716740              # starfield
# game_name = 'starfield'             # also the folder name where the reviews are stored

game_steamid = 1118010
game_name = 'monster_hunter_world_iceborne'

# game_steamid = 582010
# game_name = 'monster_hunter_world'

# game_steamid = 2138330          # cyberpunk2077 phantom liberty
# game_name = 'cyberpunk2077_phantom_liberty'

# game_steamid = 1091500          # cyberpunk2077
# game_name = 'cyberpunk2077'

# game_steamid = 730
# game_name = 'counter-strike_2'

# game_steamid = 570
# game_name = "dota2"


In [32]:
# load the reviews from folder

reviews_reqs = []

# get existing folder and retrieve the cursor object (?)

# load the latest file
game_folder = Path(f'../../dataset/data_scraping/steam_comments_scraping/{game_name}')
if game_folder.exists():
    try:
        latest_file_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis.pkl')
        with open(latest_file_path, 'rb') as f:
            reviews_reqs = pickle.load(f)           # retrieve the list of reviews
            print('Loaded:', latest_file_path)
    except IndexError as e:
        print('Error loading the latest file:', e)
        traceback.print_exc()

Loaded: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis.pkl


In [33]:
reviews_reqs[0]

{'recommendationid': '159116573',
 'author': {'steamid': '76561199275802470',
  'num_games_owned': 0,
  'num_reviews': 1,
  'playtime_forever': 0,
  'playtime_last_two_weeks': 0,
  'last_played': 0},
 'language': 'english',
 'review': 'Finally Imma try this game.',
 'timestamp_created': 1708748869,
 'timestamp_updated': 1708748869,
 'voted_up': True,
 'votes_up': 0,
 'votes_funny': 0,
 'weighted_vote_score': 0,
 'comment_count': 0,
 'steam_purchase': False,
 'received_for_free': False,
 'written_during_early_access': False,
 'hidden_in_steam_china': True,
 'steam_china_location': '',
 'author_generated': {'name': 'rsantiago468',
  'email': 'rsantiago468@yandex.ru',
  'password': 'rsantiago468',
  'birthdate': '2003-07-07',
  'gender': 'MALE'},
 'analysis': {'SA': 1, 'TM_topic_id': -1}}

In [34]:
# also load the topic frequency table

topic_freq_path = game_folder.joinpath(
    f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_topic_freq.csv'
)

topic_freq = pd.read_csv(topic_freq_path, index_col=None)

Create different tables

- genderReviews
- ageReviews


Using sentiment analysis results

- sentimentReviews
- sentimentReviewsByGender
- sentimentReviewsByAge


In [35]:
MIN_MAX_AGE = {
    '13-19': (13, 19),
    '20-29': (20, 29),
    '30-39': (30, 39),
    '40-49': (40, 49),
    '50-59': (50, 59),
    '60-69': (60, 69),
    '70-79': (70, 79),
    '80-89': (80, 89),
    '90-99': (90, 99)
}

AGE_GROUPS = list(MIN_MAX_AGE.keys()) + ["N/A"]
AGE_RANGE_TO_AGE_GROUP = {v: k for k, v in MIN_MAX_AGE.items()}

GENDERS = ['MALE', 'FEMALE', "OTHER", "UNDISCLOSED", "N/A"]

In [36]:
analysis_date = datetime(2024, 3, 22)       # or get latest date using datetime.now().today()

In [37]:
def get_age_range(review:json):
    bod = review['author_generated']['birthdate']
    bod = datetime.strptime(bod, '%Y-%m-%d')
    age = analysis_date.year - bod.year - ((analysis_date.month, analysis_date.day) < (bod.month, bod.day))

    for age_group, (min_age, max_age) in MIN_MAX_AGE.items():
        if min_age <= age <= max_age:
            return age_group
        
    return "N/A"        # default value in the API

In [38]:
# create df for ageReviews

ageReviews_json = {}
for age_group in AGE_GROUPS:
    ageReviews_json[age_group] = 0

for review in reviews_reqs:
    age_group = get_age_range(review)
    ageReviews_json[age_group] += 1

ageReviews_df = pd.DataFrame(ageReviews_json.items(), columns=['age_group', 'count'])
ageReviews_df

,age_group,count
0,13-19,1717
1,20-29,2076
2,30-39,1768
3,40-49,684
4,50-59,332
5,60-69,333
6,70-79,0
7,80-89,0
8,90-99,0
9,N/A,0


In [39]:
# create df for genderReviews

genderReviews_json = {k:0 for k in GENDERS}

for review in reviews_reqs:
    genderReviews_json[review['author_generated']['gender']] += 1

genderReviews_df = pd.DataFrame(genderReviews_json.items(), columns=['gender', 'count'])
genderReviews_df

,gender,count
0,MALE,5563
1,FEMALE,1347
2,OTHER,0
3,UNDISCLOSED,0
4,N/A,0


In [40]:
# create sentimentReviews

sentimentReviews_json = {k:0 for k in ['POSITIVE', 'NEGATIVE', "N/A"]}
for review in reviews_reqs:
    sentimentReviews_json["POSITIVE" if review['analysis']['SA'] else "NEGATIVE"] += 1        # 1 for positive, 0 for negative

sentimentReviews_df = pd.DataFrame(sentimentReviews_json.items(), columns=['sentiment', 'count'])
sentimentReviews_df

# and the actual sentiment count

sentimentReviews_truelabel_json = {k:0 for k in ['POSITIVE', 'NEGATIVE', "N/A"]}
for review in reviews_reqs:
    sentimentReviews_truelabel_json["POSITIVE" if review['voted_up'] else "NEGATIVE"] += 1      # "voted_up" is the actual sentiment, and stores a boolean value

sentimentReviews_truelabel_df = pd.DataFrame(sentimentReviews_truelabel_json.items(), columns=['sentiment', 'count'])
sentimentReviews_truelabel_df

,sentiment,count
0,POSITIVE,5429
1,NEGATIVE,1481
2,N/A,0


In [41]:
from itertools import product

sentimentByAgeGroup_df_template = pd.DataFrame(product(
    AGE_GROUPS, ['POSITIVE', 'NEGATIVE', "N/A"], [0]
), columns=['age_group', 'sentiment', 'count'])

sentimentByAgeGroup_df_template

,age_group,sentiment,count
0,13-19,POSITIVE,0
1,13-19,NEGATIVE,0
2,13-19,N/A,0
3,20-29,POSITIVE,0
4,20-29,NEGATIVE,0
5,20-29,N/A,0
6,30-39,POSITIVE,0
7,30-39,NEGATIVE,0
8,30-39,N/A,0
9,40-49,POSITIVE,0


In [42]:
# sentiment by age group

sentimentByAgeGroup = []

for review in reviews_reqs:
    # get the age group of the review
    age_group = get_age_range(review)
    sentiment = "POSITIVE" if review['analysis']['SA'] else "NEGATIVE"
    sentimentByAgeGroup.append((age_group, sentiment))

# count the number of reviews for each (age_group, sentiment) pair
sentimentByAgeGroup_df = pd.DataFrame(sentimentByAgeGroup, columns=['age_group', 'sentiment'])
sentimentByAgeGroup_df = sentimentByAgeGroup_df.groupby(['age_group', 'sentiment']).size().reset_index(name='count')
# reorder columns
sentimentByAgeGroup_df = sentimentByAgeGroup_df[['sentiment', 'age_group', 'count']]

# update with the template
sentimentByAgeGroup_df = sentimentByAgeGroup_df_template.merge(sentimentByAgeGroup_df, on=['age_group', 'sentiment'], how='left')
sentimentByAgeGroup_df['count'] = np.max(sentimentByAgeGroup_df[['count_x', 'count_y']], axis=1)
sentimentByAgeGroup_df = sentimentByAgeGroup_df[['sentiment', 'age_group', 'count']]

# drop rows with count = 0
sentimentByAgeGroup_df = sentimentByAgeGroup_df[sentimentByAgeGroup_df['count'] > 0]

# sort by sentiment
sentimentByAgeGroup_df = sentimentByAgeGroup_df.sort_values(by=['sentiment', 'age_group'])
sentimentByAgeGroup_df.reset_index(drop=True, inplace=True)


sentimentByAgeGroup_df

,sentiment,age_group,count
0,NEGATIVE,13-19,479.0
1,NEGATIVE,20-29,606.0
2,NEGATIVE,30-39,518.0
3,NEGATIVE,40-49,207.0
4,NEGATIVE,50-59,98.0
5,NEGATIVE,60-69,94.0
6,POSITIVE,13-19,1238.0
7,POSITIVE,20-29,1470.0
8,POSITIVE,30-39,1250.0
9,POSITIVE,40-49,477.0


In [43]:
sentimentByGender_df_template = pd.DataFrame(product(
    GENDERS, ['POSITIVE', 'NEGATIVE', "N/A"], [0]
), columns=['gender', 'sentiment', 'count'])

sentimentByGender_df_template

,gender,sentiment,count
0,MALE,POSITIVE,0
1,MALE,NEGATIVE,0
2,MALE,N/A,0
3,FEMALE,POSITIVE,0
4,FEMALE,NEGATIVE,0
5,FEMALE,N/A,0
6,OTHER,POSITIVE,0
7,OTHER,NEGATIVE,0
8,OTHER,N/A,0
9,UNDISCLOSED,POSITIVE,0


In [44]:
# sentiment by gender

sentimentByGender = []

for review in reviews_reqs:
    # get the gender of the review
    gender = review['author_generated']['gender']
    sentiment = "POSITIVE" if review['analysis']['SA'] else "NEGATIVE"
    
    sentimentByGender.append([gender, sentiment])

# count the number of reviews for each (gender, sentiment) pair
sentimentByGender_df = pd.DataFrame(sentimentByGender, columns=['gender', 'sentiment'])
sentimentByGender_df = sentimentByGender_df.groupby(['gender', 'sentiment']).size().reset_index(name='count')

# reorder columns
sentimentByGender_df = sentimentByGender_df[['sentiment', 'gender', 'count']]

# update with the template
sentimentByGender_df = sentimentByGender_df_template.merge(sentimentByGender_df, on=['sentiment', 'gender'], how='left')
sentimentByGender_df['count'] = np.max(sentimentByGender_df[['count_x', 'count_y']], axis=1)
sentimentByGender_df = sentimentByGender_df[['sentiment', 'gender', 'count']]

# drop rows with count = 0
sentimentByGender_df = sentimentByGender_df[sentimentByGender_df['count'] > 0]

# sort by sentiment
sentimentByGender_df = sentimentByGender_df.sort_values(by=['sentiment', 'gender'])
sentimentByGender_df.reset_index(drop=True, inplace=True)


sentimentByGender_df


,sentiment,gender,count
0,NEGATIVE,FEMALE,396.0
1,NEGATIVE,MALE,1606.0
2,POSITIVE,FEMALE,951.0
3,POSITIVE,MALE,3957.0


In [45]:
# save each csv
save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_ageGroup.csv')
ageReviews_df.to_csv(save_path, index=False)
print('Saved:', save_path)

save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_genderReviews.csv')
genderReviews_df.to_csv(save_path, index=False)
print('Saved:', save_path)

save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews.csv')
sentimentReviews_df.to_csv(save_path, index=False)
print('Saved:', save_path)

save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentReviews_truelabel.csv')
sentimentReviews_truelabel_df.to_csv(save_path, index=False)
print('Saved:', save_path)

save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByAgeGroup.csv')
sentimentByAgeGroup_df.to_csv(save_path, index=False)
print('Saved:', save_path)

save_path = game_folder.joinpath(f'steam_reviews_{game_steamid}_unique_with_gendata_with_analysis_sentimentByGender.csv')
sentimentByGender_df.to_csv(save_path, index=False)
print('Saved:', save_path)

Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_ageGroup.csv
Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_genderReviews.csv
Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_sentimentReviews.csv
Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_sentimentReviews_truelabel.csv
Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_sentimentByAgeGroup.csv
Saved: ../../dataset/data_scraping/steam_comments_scraping/monster_hunter_world_iceborne/steam_reviews_1118010_unique_with_gendata_with_analysis_sentimentByGender.csv


---

Bonus

Try to pull from FYP API and create the dataframes (except the topic modeling one)

In [46]:
# REFERENCE: game name to id mapping
GAME_NAME_TO_ID = {
    'Counter-Strike 2':1,
    'Monster Hunter World':4,
    'Cyberpunk 2077': 5,
    'Cyberpunk 2077 Phantom Liberaty': 7,
    'Starfield': 8,
    'Dota 2': 9,
    'Monster Hunter World: Iceborne': 30
}

In [47]:
import requests


IP = 'https://critiqbackend.itzjacky.info'
PORT = 9000
api = f'{IP}:{PORT}/api/game/gameAnalytic'

# send the data to the backend
req = requests.post(api, json={"id":7})
return_json = req.json()

return_json

{'favouriteByAge': {'20-29': 1,
  'N/A': 2,
  '30-39': 0,
  '40-49': 0,
  '50-59': 0,
  '13-19': 0,
  '60-69': 0,
  '70-79': 0,
  '80-89': 0,
  '90-99': 0},
 'numberOfReviews': 13,
 'recommendedReviews': {'DO NOT RECOMMEND': 1, 'RECOMMEND': 12},
 'description': 'Cyberpunk RPG',
 'developerCompany': 'CDProjekt Red',
 'genderReviews': {'OTHER': 0,
  'UNDISCLOSED': 1,
  'N/A': 6,
  'MALE': 6,
  'FEMALE': 0},
 'reviewLength': {'200-300': 0,
  '500-600': 0,
  '600-700': 0,
  '900-1000': 0,
  '100-200': 0,
  '1000+': 1,
  '300-400': 0,
  '400-500': 0,
  '700-800': 0,
  '800-900': 0,
  '0-100': 12},
 'sentimentReviewsByGender': {'POSITIVE': {'OTHER': 0,
   'UNDISCLOSED': 1,
   'N/A': 5,
   'MALE': 6,
   'FEMALE': 0},
  'NEGATIVE': {'OTHER': 0,
   'UNDISCLOSED': 0,
   'N/A': 1,
   'MALE': 0,
   'FEMALE': 0}},
 'score': 83.1538,
 'isInDevelopment': False,
 'playTime': {'1-20 Hours': 8,
  'N/A': 0,
  '50-100 Hours': 0,
  '100+ Hours': 0,
  '<1 Hour': 2,
  '20-50 Hours': 3},
 'id': 7,
 'iconUrl':

In [48]:
# create df for ageReviews
import pandas as pd

ageReviews_json = {}
ageReviews_from_api = return_json['ageReviews']
for k, v in ageReviews_from_api.items():
    ageReviews_json[k] = v

ageReviews_df = pd.DataFrame(ageReviews_json.items(), columns=['age_group', 'count'])
ageReviews_df.sort_values(by='age_group', inplace=True)
ageReviews_df.reset_index(drop=True, inplace=True)

ageReviews_df

,age_group,count
0,13-19,0
1,20-29,7
2,30-39,0
3,40-49,0
4,50-59,0
5,60-69,0
6,70-79,0
7,80-89,0
8,90-99,0
9,N/A,6


In [49]:
# create df for genderReviews

genderReviews_json = {k:0 for k in GENDERS}     # define the ordering in genderReviews_df
genderReviews_from_api = return_json['genderReviews']
for k, v in genderReviews_from_api.items():
    genderReviews_json[k] = v

genderReviews_df = pd.DataFrame(genderReviews_json.items(), columns=['gender', 'count'])
genderReviews_df

,gender,count
0,MALE,6
1,FEMALE,0
2,OTHER,0
3,UNDISCLOSED,1
4,N/A,6


In [50]:
# create sentimentReviews_df

sentimentReviews_json = {k:0 for k in ['POSITIVE', 'NEGATIVE', "N/A"]}
sentimentReviews_from_api = return_json['sentimentReviews']
for k, v in sentimentReviews_from_api.items():
    sentimentReviews_json[k] = v

sentimentReviews_df = pd.DataFrame(sentimentReviews_json.items(), columns=['sentiment', 'count'])
sentimentReviews_df

,sentiment,count
0,POSITIVE,12
1,NEGATIVE,1
2,N/A,0


In [51]:
# create sentimentByAgeGroup_df

# get all possible combination of (age_group, sentiment)
sentimentByAgeGroup = []

sentimentByAgeGroup_from_api = return_json['sentimentReviewsByAge']
for sentiment, sentiment_json in sentimentByAgeGroup_from_api.items():
    for age_group, count in sentiment_json.items():
        sentimentByAgeGroup.append((age_group, sentiment, count))

# create a three column df
sentimentByAgeGroup_df = pd.DataFrame(sentimentByAgeGroup, columns=['age_group', 'sentiment', 'count'])
# reorder columns
sentimentByAgeGroup_df = sentimentByAgeGroup_df[['sentiment', 'age_group', 'count']]
# sort by sentiment, then age_group
sentimentByAgeGroup_df = sentimentByAgeGroup_df.sort_values(by=['sentiment', 'age_group'])

# drop row with count = 0
sentimentByAgeGroup_df = sentimentByAgeGroup_df[sentimentByAgeGroup_df['count'] > 0]

sentimentByAgeGroup_df

,sentiment,age_group,count
11,NEGATIVE,N/A,1
0,POSITIVE,20-29,7
1,POSITIVE,N/A,5


In [52]:
sentimentByGender_df_template = pd.DataFrame(product(
    GENDERS, ['POSITIVE', 'NEGATIVE', "N/A"], [0]
), columns=['gender', 'sentiment', 'count'])

sentimentByGender_df_template

,gender,sentiment,count
0,MALE,POSITIVE,0
1,MALE,NEGATIVE,0
2,MALE,N/A,0
3,FEMALE,POSITIVE,0
4,FEMALE,NEGATIVE,0
5,FEMALE,N/A,0
6,OTHER,POSITIVE,0
7,OTHER,NEGATIVE,0
8,OTHER,N/A,0
9,UNDISCLOSED,POSITIVE,0


In [53]:
# sentimentByGender_df

# get all possible combination of (gender, sentiment)
sentimentByGender = []

for sentiment, sentiment_json in return_json['sentimentReviewsByGender'].items():
    for gender, count in sentiment_json.items():
        sentimentByGender.append((sentiment, gender, count))

# create a three column df
sentimentByGender_df = pd.DataFrame(sentimentByGender, columns=['sentiment','gender', 'count'])
# sort by sentiment then gender
sentimentByGender_df = sentimentByGender_df.sort_values(by=['sentiment', 'gender'])
# drop rows with count = 0
sentimentByGender_df = sentimentByGender_df[sentimentByGender_df['count'] > 0]

sentimentByGender_df

,sentiment,gender,count
7,NEGATIVE,N/A,1
3,POSITIVE,MALE,6
2,POSITIVE,N/A,5
1,POSITIVE,UNDISCLOSED,1
